## Supplement 4: Classification

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd


### 4.1 Programming Task: Gaussian Naive-Bayes Classifier
The Iris dataset, containing measurements of the flower parts obtained from 3 different species of the Iris plant, is provided in the file __iris.csv__. The first four columns of the dataset contain the measurement values representing input features for the model and the last column contains class labels of the plant species: Iris-setosa, Iris-versicolor, and Iris-virginica.
The goal of this task is to implement a Gaussian Naive-Bayes classifier for the Iris dataset.

i\. What are the assumptions on the dataset required for the Gaussian Naive-Bayes model?

-features are independent (diagonal covariance matrix)

ii\. Split the dataset into train and test by the 80:20 ratio.


iii\. Estimate the parameters of the Gaussian Naive-Bayes classifier using the train set.


iv\. Using the learned parameters, predict the classes for the samples in the test set.


What is the accuracy of the model on the test set?